#### Visualization of Xenium healthy gut data after the labels transer 
- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Date:** 30th January 2024

+ Import Packages

In [ ]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
from scipy.io import mmread
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci
from scipy.sparse import coo_matrix
from PIL import Image

In [ ]:
from libtiff import TIFF

+ Upload anndata objects

In [ ]:
# upload h5 object
adata_xenium = sc.read_10x_h5('/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/outs/cell_feature_matrix.h5')

In [ ]:
input = '/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/epithelial_scRNA_and_Xenium_data_scANVI.h5ad'
adata = sc.read_h5ad(input)

In [ ]:
input = '/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/scRNA_and_Xenium_data_scANVI.h5ad'
adata_full = sc.read_h5ad(input)

+ Upload images

In [ ]:
tif = TIFF.open('/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.ome.tif')

In [ ]:
image = tif.read_image()

+ Upload coordinates

In [ ]:
coords = pd.read_csv(
        "/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/outs/cells.csv",
        header=0)

In [ ]:
# rename index column in adata_xenium to 'cell_id'
adata_xenium.obs.index.name = 'cell_id'

In [ ]:
adata_xenium.obs = adata_xenium.obs.merge(coords, on='cell_id', how='left')

In [ ]:
adata_xenium.obs

In [ ]:
adata_xenium.obsm["spatial"] = adata_xenium.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

In [ ]:
nucleus_boundaries = pd.read_csv(
        "/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/outs/nucleus_boundaries.csv",
        header=0)

In [ ]:
spatial_key = "spatial"
library_id = "tissue42"
adata_xenium.uns[spatial_key] = {library_id: {}}
adata_xenium.uns[spatial_key][library_id]["images"] = {}
adata_xenium.uns[spatial_key][library_id]["images"] = {"image": image}

+ Add predicted cell types and cell states to adata_xenium

In [ ]:
adata.obs.index.name = 'cell_id'

In [ ]:
adata_full.obs.index.name = 'cell_id'

In [ ]:
adata_xenium.obs = adata_xenium.obs.merge(adata_full.obs['C_scANVI'], on='cell_id', how='left')

In [ ]:
# rename adata_xenium.obs['C_scANVI'] to 'Cell_Type'
adata_xenium.obs.rename(columns={'C_scANVI': 'Cell_Type'}, inplace=True)

In [ ]:
adata_xenium.obs = adata_xenium.obs.merge(adata.obs['C_scANVI'], on='cell_id', how='left')

In [ ]:
adata_xenium.obs.rename(columns={'C_scANVI': 'Cell_States'}, inplace=True)

+ Write anndata

In [ ]:
adata_xenium.write_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/Xenium_Non-diseased_pre-designed_pallet_with_predicted_epi_labels.h5ad')

In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    shape=None,
    color=[
        "Cell_Type",
    ],
    wspace=0.4,
)

In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    shape=None,
    color=[
        "Cell_States",
    ],
    wspace=0.4,
)

+ Visualize stem cells

In [ ]:
input = '/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/Xenium_Non-diseased_pre-designed_pallet_with_predicted_epi_and_SC_labels.h5ad'
adata_xenium = sc.read_h5ad(input)

In [ ]:
adata_xenium.obs

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10), dpi=300)
plt.imshow(image)  
plt.axis('off')  

In [ ]:
adata_xenium.obs['stem_cell_type'] = adata_xenium.obs['stem_cell_type'].cat.add_categories(['Not_SC'])

adata_xenium.obs['stem_cell_type'] = adata_xenium.obs['stem_cell_type'].fillna('Not_SC')

+ Normalize and log transform the dataset

In [ ]:
sc.pp.normalize_total(adata_xenium, target_sum = 1e6, exclude_highly_expressed = True)
sc.pp.log1p(adata_xenium)

+ Identify top 10 differentially expressed genes

In [ ]:
sc.tl.rank_genes_groups(adata_xenium, groupby='stem_cell_type', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups(adata_xenium, n_genes=25, sharey=False)

In [ ]:
adata_xenium.obs['Cell_States'] = adata_xenium.obs['Cell_States'].cat.add_categories(['Not_epithelial'])

adata_xenium.obs['Cell_States'] = adata_xenium.obs['Cell_States'].fillna('Not_epithelial')

In [ ]:
sc.tl.rank_genes_groups(adata_xenium, groupby='Cell_States', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups(adata_xenium, n_genes=25, sharey=False)

+ Visualize markers

In [ ]:
adata_xenium.var

In [ ]:
# Check if DEFA1 is in var_names 
'MUC4' in adata_xenium.var_names

In [ ]:
sc.set_figure_params(dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    shape=None,
    color=[
        "Cell_States", 'SPDEF', 'BCAS1', 'CLCA1', 
    ],
    wspace=0.4,
)